# Recurrent Neural Network in 2D and 3D lightcones

There are several ideas we would like to explore, concerning RNN networks.

1. Compessing spacial data to N parameters with CNN, with network weights independent of time (redshift) and feeding those to RNN network which in every step tries to predict astrophysical parameters, and then weight somehow the sum all of those in the loss function.
2. Using the whole CNN as a part of RNN network which updates it's weights in each iteration
3. Some hybrid of usual convolution in spacial dimensions and WaveNet-like convolutions in time dimension

In 1. and 2. for RNN part, we use LSTM cells

## Things to try
- both directions of learning, bidirectional LSTM?

In [1]:
import numpy as np
import sys
import os
import json
import tensorflow as tf
import keras
# os.environ['KERAS_BACKEND'] = 'tensorflow'


# #setting up GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 1. #setting the percentage of GPU usage
config.gpu_options.visible_device_list = "0" #for picking only some devices
# config.gpu_options.allow_growth = True
config.log_device_placement=True
from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config)) 

from keras import backend as K
K.set_image_data_format('channels_last')

/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/media/yqin/81614149-2ed8-4c76-82e1-c46763d086fa/pregi/anaconda3/envs/21CNN_GPU_NEW/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it

In [2]:
#working with sliced data shape (10000, 4, 25, 25, ~500)
DataLoc = "../data/"
XName = "data3D_boxcar444_sliced22_float32.npy"
YName = "databaseParams_float32.npy"
YbackupName = "databaseParams_min_max.txt"

In [3]:
X = {}
Y = {}

In [11]:
for sort in ["train", "val", "test"]:
    X[sort] = np.load(f"{DataLoc}{sort}/X_{XName}")
    X[sort] = X[sort][..., np.newaxis]
#     print(np.amin(X[sort]), np.amax(X[sort]))

In [4]:
# X['train'] = X['train'][..., np.newaxis]
# X['train'].shape

In [5]:
# #correcting data just once, old error, don't run twice
# for sort in ["train", "val", "test"]:
#     X[sort] = np.squeeze(X[sort], axis=-1)
# #     print(np.amin(X[sort]), np.amax(X[sort]))
#     print(X[sort].shape)
# #     X[sort] /=4
#     np.save(f"{DataLoc}{sort}/X_{XName}", X[sort])
#     X[sort] = X[sort][..., np.newaxis]

In [6]:
# for sort in ["train", "val", "test"]:
#     print(X[sort].shape)

(32000, 526, 25, 25, 1)
(4000, 526, 25, 25, 1)
(4000, 526, 25, 25, 1)


In [12]:
for sort in ["train", "val", "test"]:
    Y[sort] = np.load(f"{DataLoc}{sort}/Y_{XName}")
    print(Y[sort].shape)
#     print(np.amin(Y[sort]), np.amax(Y[sort]))
    shape = Y[sort].shape
    Y[sort] = Y[sort][..., np.newaxis]
    Y[sort] = np.broadcast_to(Y[sort], shape + (X[sort].shape[1],))
    Y[sort] = np.swapaxes(Y[sort], -1, -2)
    print(Y[sort].shape)

(32000, 4)
(32000, 526, 4)
(4000, 4)
(4000, 526, 4)
(4000, 4)
(4000, 526, 4)


In [31]:
Y_basic = {}
for sort in ["train", "val", "test"]:
    Y_basic[sort] = np.load(f"{DataLoc}{sort}/Y_{XName}")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
EoR_colour = matplotlib.colors.LinearSegmentedColormap.from_list('mycmap', [(0, 'white'),(0.33, 'yellow'),(0.5, 'orange'),(0.68, 'red'),(0.833, 'black'),(0.87, 'blue'),(1, 'cyan')])
plt.register_cmap(cmap=EoR_colour)

In [ ]:
for i in range(10):
    print(Y['test'][i])
    fig=plt.figure(figsize=(10, 1))
    plt.pcolormesh(X['test'][i, :, :, 0].T, vmin = 0, vmax = 1, cmap=EoR_colour,shading='gouraud')
    plt.show()

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, CuDNNLSTM, TimeDistributed, Dense
from keras.layers import Conv3D, Conv2D, Conv1D, GlobalAveragePooling2D
from keras.layers import ConvLSTM2D, MaxPooling2D, MaxPooling3D
from keras.layers import BatchNormalization, Flatten, Dropout

In [6]:
class HyperParams1:
    def __init__(
        self,
        LossDecay = 0.95,
        LossLength = 50,
        SummaryDimension = 16,
        ):
        self.LossDecay = LossDecay
        self.LossLength = LossLength
        self.SummaryDimension = SummaryDimension
    
    def LossFunction(self):
        def loss(y_true, y_pred):
            true = y_true[..., -self.LossLength:, :]
            pred = y_pred[..., -self.LossLength:, :]
            weights = self.LossDecay**np.arange(self.LossLength, 0, -1)
#             LossValue = np.sum(weights * np.linalg.norm(predicted - true, axis=-1))
            
            return tf.mean(weights * tf.square(pred-true), axis=-1)
        return loss
    
# tf.keras.metrics.mean_squared_error()

In [7]:
HP = HyperParams1(LossDecay=0.9, LossLength=5, SummaryDimension=64)
# input_shape = X["train"].shape[1:]
input_shape = (526, 25, 25, 1)

In [8]:
from keras import backend as K
class CoeffDetermination:
    def __init__(
        self,
        timestep = 1,
        ):
        self.timestep = timestep
#         assert self.timestep <= HP.LossLength
    
    def timestep_cd(self, y_true, y_pred):
        true = y_true[..., -self.timestep, :]
        pred = y_pred[..., -self.timestep, :]
    #     SS_res = (y_true - y_pred[..., -1, :]) @ (y_true - y_pred[..., -1, :])
    #     SS_tot = (y_true - np.mean(y_true)) @  (y_true - np.mean(y_true))
    #     return (1 - SS_res/(SS_tot + 1e-7))

        SS_res =  K.sum(K.square( true-pred )) 
        SS_tot = K.sum(K.square( true - K.mean(true) ) )
#         loss = tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1)
        return ( 1 - SS_res/(SS_tot + K.epsilon()))
    def timestep_loss(self, y_true, y_pred):
        true = y_true[..., -self.timestep, :]
        pred = y_pred[..., -self.timestep, :]
        loss = tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1)
        return loss
    
def coef_determination(y_true, y_pred):
        SS_res =  K.sum(K.square( y_true-y_pred )) 
        SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
#         loss = tf.keras.backend.mean(tf.keras.backend.square(y_pred - y_true), axis=-1)
        return ( 1 - SS_res/(SS_tot + K.epsilon()))
    
# opt = keras.optimizers.RMSprop(lr=0.1)
### DEFINE THE LEARNING RATE
factor=0.5
patience=5

### set the learning rate callback
callbacks_list=[]
from keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
lrate = ReduceLROnPlateau( monitor='loss', factor=factor, patience=patience )
callbacks_list.append( lrate )

### to print the Learning Rate
from keras.callbacks import Callback, EarlyStopping
class LR_tracer(Callback):
    def on_epoch_begin(self, epoch, logs={}):
        lr = K.eval( self.model.optimizer.lr )
        print( ' LR: %.10f '%(lr) )
callbacks_list.append( LR_tracer() )

### STOP when it stop to learn
early_stopping = EarlyStopping(monitor='loss', min_delta=0.0001, patience=15, verbose=True, mode='auto')
callbacks_list.append( early_stopping )

In [9]:
def Loss1(y_true, y_pred):
    length = input_shape[0]
    weights = HP.LossDecay**np.arange(HP.LossLength - 1, -1, -1)
    weights = np.append(np.zeros(length-HP.LossLength), weights)
    weights = np.broadcast_to(weights[..., np.newaxis], (length, 4))
#     weights = tf.math.pow(tf.constant([HP.LossDecay], dtype=tf.float32), tf.range(tf.shape(y_true)[-2] - 1, -1, -1, dtype=tf.float32))
#     weights = HP.LossDecay**np.arange(np.array(y_true).shape[-2] - 1, -1, -1)
    weights = tf.convert_to_tensor(weights, dtype=tf.float32)
    sq = weights * tf.keras.backend.square(y_pred - y_true)
    return tf.keras.backend.mean(sq, axis=-1)

In [13]:
opt = keras.optimizers.RMSprop(lr=0.001)
# model1.compile( optimizer='adam', 
#                 loss=HP.LossFunction())
model1.compile( optimizer=opt, 
                loss=Loss1,
                metrics=[CoeffDetermination(500).timestep_cd, CoeffDetermination(1).timestep_cd, CoeffDetermination(500).timestep_loss, CoeffDetermination(1).timestep_loss])

In [ ]:
model1 = Sequential()

#convolutions
model1.add(TimeDistributed(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'), input_shape = input_shape, name="Conv0"))


for i in range(4):
    model1.add(TimeDistributed(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'), name=f"Conv{i+1}"))
model1.add(TimeDistributed(Conv2D(filters=HP.SummaryDimension, kernel_size=(5, 5), activation='relu'), name="ConvSummary"))

model1.add(TimeDistributed(Flatten(), name="Flatten"))
# model1.add(TimeDistributed(Dense(HP.SummaryDimension)))
model1.add(TimeDistributed(Dropout(0.2), name="Dropout"))

#RNN part
model1.add(CuDNNLSTM(32, return_sequences=True, name="DenseLSTM0"))
model1.add(CuDNNLSTM(32, return_sequences=True, name="DenseLSTM1"))
model1.add(CuDNNLSTM(16, return_sequences=True, name="DenseLSTM2"))
model1.add(CuDNNLSTM(16, return_sequences=True, name="DenseLSTM3"))
model1.add(CuDNNLSTM(8, return_sequences=True, name="DenseLSTM4"))
# model1.add(TimeDistributed(Dense(16, activation='relu'), name="Dense1"))
model1.add(TimeDistributed(Dense(4), name="out"))

# model1.add(Dense(4))
model1.summary()

In [16]:
history = model1.fit(  X['train'], Y['train'],
                     epochs=200,
                     batch_size=20,
                     callbacks=callbacks_list,
                     validation_data=(X['val'], Y['val']),
                     verbose=True,
                     )


Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0010000000 
32000/32000 [==============================] - 976s 30ms/step - loss: 6.5437e-04 - timestep_cd: -0.0272 - timestep_cd_1: -0.0270 - timestep_loss: 0.0854 - timestep_loss_1: 0.0854 - val_loss: 6.4757e-04 - val_timestep_cd: -0.0186 - val_timestep_cd_1: -0.0186 - val_timestep_loss: 0.0841 - val_timestep_loss_1: 0.0841
Epoch 2/200
 LR: 0.0010000000 
32000/32000 [==============================] - 913s 29ms/step - loss: 6.4725e-04 - timestep_cd: -0.0166 - timestep_cd_1: -0.0166 - timestep_loss: 0.0839 - timestep_loss_1: 0.0839 - val_loss: 6.4595e-04 - val_timestep_cd: -0.0160 - val_timestep_cd_1: -0.0160 - val_timestep_loss: 0.0837 - val_timestep_loss_1: 0.0837
Epoch 3/200
 LR: 0.0010000000 
32000/32000 [==============================] - 941s 29ms/step - loss: 6.4727e-04 - timestep_cd: -0.0166 - timestep_cd_1: -0.0166 - timestep_loss: 0.0838 - timestep_loss_1: 0.0838 - val_loss: 6.4397e-04 - val_timestep_cd: -0.0

In [29]:
model1_simple = Sequential()
model1_simple.add(TimeDistributed(Conv2D(filters=128, kernel_size=(11, 11)), input_shape = input_shape, name="Conv0"))
model1_simple.add(TimeDistributed(GlobalAveragePooling2D(), name="GlobalPool"))
model1_simple.add(TimeDistributed(Flatten(), name="Flatten"))
model1_simple.add(TimeDistributed(Dropout(0.2), name="Dropout"))

#RNN part
model1_simple.add(CuDNNLSTM(64, return_sequences=True, name="DenseLSTM0"))
model1_simple.add(CuDNNLSTM(32, return_sequences=True, name="DenseLSTM1"))
model1_simple.add(CuDNNLSTM(16, return_sequences=True, name="DenseLSTM2"))
model1_simple.add(TimeDistributed(Dense(4), name="out"))
# model1.add(Dense(4))
model1_simple.summary()

opt = keras.optimizers.RMSprop(lr=0.1)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (TimeDistributed)      (None, 526, 15, 15, 128)  15616     
_________________________________________________________________
GlobalPool (TimeDistributed) (None, 526, 128)          0         
_________________________________________________________________
Flatten (TimeDistributed)    (None, 526, 128)          0         
_________________________________________________________________
Dropout (TimeDistributed)    (None, 526, 128)          0         
_________________________________________________________________
DenseLSTM0 (CuDNNLSTM)       (None, 526, 64)           49664     
_________________________________________________________________
DenseLSTM1 (CuDNNLSTM)       (None, 526, 32)           12544     
___________________________

In [30]:
model1_simple.compile( optimizer=opt, 
                loss=Loss1,
                metrics=[CoeffDetermination(1).timestep_cd, CoeffDetermination(1).timestep_loss])

In [31]:
history_simple = model1_simple.fit(X['train'], Y['train'],
                 epochs=200,
                 batch_size=20,
         callbacks=callbacks_list,
                 validation_data=(X['val'], Y['val']),
                 verbose=True,
                 )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.1000000015 
32000/32000 [==============================] - 604s 19ms/step - loss: 7.5112e-04 - cd: -0.1880 - cd_loss: 0.0965 - val_loss: 6.9999e-04 - val_cd: -0.1014 - val_cd_loss: 0.0899
Epoch 2/200
 LR: 0.1000000015 
32000/32000 [==============================] - 421s 13ms/step - loss: 6.8029e-04 - cd: -0.0692 - cd_loss: 0.0874 - val_loss: 6.7949e-04 - val_cd: -0.0690 - val_cd_loss: 0.0873
Epoch 3/200
 LR: 0.1000000015 
32000/32000 [==============================] - 338s 11ms/step - loss: 6.8093e-04 - cd: -0.0699 - cd_loss: 0.0875 - val_loss: 6.5586e-04 - val_cd: -0.0322 - val_cd_loss: 0.0842
Epoch 4/200
 LR: 0.1000000015 
32000/32000 [==============================] - 296s 9ms/step - loss: 6.8013e-04 - cd: -0.0683 - cd_loss: 0.0874 - val_loss: 6.7317e-04 - val_cd: -0.0593 - val_cd_loss: 0.0865
Epoch 5/200
 LR: 0.1000000015 
32000/32000 [==============================] - 292s 9ms/step - loss: 6.8048e-04 - cd: -0.0685

KeyboardInterrupt: 

In [32]:
model1_proper = Sequential(
    [
        TimeDistributed(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'), input_shape = input_shape, name="Conv0"),
        TimeDistributed(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'), name="Conv1"),
        TimeDistributed(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'), name="Conv2"),
        TimeDistributed(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'), name="Conv3"),
        TimeDistributed(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'), name="Conv4"),
        TimeDistributed(Conv2D(filters=256, kernel_size=(5, 5), activation='relu'), name="Conv5"),
        TimeDistributed(Flatten(), name="Flatten"),
        TimeDistributed(Dropout(0.2), name="Dropout"),
        CuDNNLSTM(128, return_sequences=True, name="DenseLSTM0"),
        CuDNNLSTM(64, return_sequences=True, name="DenseLSTM1"),
        CuDNNLSTM(32, name="DenseLSTM2"),
        Dense(16, activation='relu', name='dense0'),
        Dense(8, activation='relu', name='dense1'),
        Dense(4, name='out')
        ]
)
model1_proper.summary()
opt = keras.optimizers.RMSprop(lr=0.01)

model1_proper.compile(  optimizer=opt, 
                        loss='mse',
                        metrics=[coef_determination])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (TimeDistributed)      (None, 526, 21, 21, 32)   832       
_________________________________________________________________
Conv1 (TimeDistributed)      (None, 526, 17, 17, 64)   51264     
_________________________________________________________________
Conv2 (TimeDistributed)      (None, 526, 13, 13, 64)   102464    
_________________________________________________________________
Conv3 (TimeDistributed)      (None, 526, 9, 9, 128)    204928    
_________________________________________________________________
Conv4 (TimeDistributed)      (None, 526, 5, 5, 128)    409728    
_________________________________________________________________
Conv5 (TimeDistributed)      (None, 526, 1, 1, 256)    819456    
_________________________________________________________________
Flatten (TimeDistributed)    (None, 526, 256)          0         
__________

In [34]:
history1_proper = model1_proper.fit( X['train'], Y_basic['train'],
                                     epochs=200,
                                     batch_size=20,
                                     callbacks=callbacks_list,
                                     validation_data=(X['val'], Y_basic['val']),
                                     verbose=True,
                                     )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0099999998 
  640/32000 [..............................] - ETA: 15:29 - loss: 0.0847 - coef_determination: -0.0156

KeyboardInterrupt: 

In [43]:
model1_proper_simple = Sequential(
    [
        TimeDistributed(Conv2D(filters=64, kernel_size=(8, 8), activation='relu'), input_shape = input_shape, name="Conv0"),
        TimeDistributed(MaxPooling2D(), name="MaxPool"),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'), name="Conv4"),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(Conv2D(filters=128, kernel_size=(5, 5), activation='relu'), name="Conv5"),
        TimeDistributed(BatchNormalization()),
        TimeDistributed(Flatten(), name="Flatten"),
        TimeDistributed(Dropout(0.2), name="Dropout"),
        CuDNNLSTM(128, return_sequences=True, name="DenseLSTM0"),
        CuDNNLSTM(64, return_sequences=True, name="DenseLSTM1"),
        CuDNNLSTM(32, name="DenseLSTM2"),
        Dense(16, activation='relu', name='dense0'),
        Dense(8, activation='relu', name='dense1'),
        Dense(4, name='out')
        ]
)
model1_proper_simple.summary()
opt = keras.optimizers.Adam(lr=0.001)

model1_proper_simple.compile(  optimizer=opt, 
                        loss='mse',
                        metrics=[coef_determination])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv0 (TimeDistributed)      (None, 526, 18, 18, 64)   4160      
_________________________________________________________________
MaxPool (TimeDistributed)    (None, 526, 9, 9, 64)     0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 526, 9, 9, 64)     256       
_________________________________________________________________
Conv4 (TimeDistributed)      (None, 526, 5, 5, 64)     102464    
_________________________________________________________________
time_distributed_18 (TimeDis (None, 526, 5, 5, 64)     256       
_________________________________________________________________
Conv5 (TimeDistributed)      (None, 526, 1, 1, 128)    204928    
_________________________________________________________________
time_distributed_19 (TimeDis (None, 526, 1, 1, 128)    512       
__________

In [44]:
history1_proper_simple = model1_proper_simple.fit(  X['train'], Y_basic['train'],
                                             epochs=200,
                                             batch_size=20,
                                             callbacks=callbacks_list,
                                             validation_data=(X['val'], Y_basic['val']),
                                             verbose=True,
                                             )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0010000000 
32000/32000 [==============================] - 394s 12ms/step - loss: 0.0550 - coef_determination: 0.3269 - val_loss: 0.0438 - val_coef_determination: 0.4642
Epoch 2/200
 LR: 0.0010000000 
32000/32000 [==============================] - 410s 13ms/step - loss: 0.0425 - coef_determination: 0.4804 - val_loss: 0.1566 - val_coef_determination: -0.9244
Epoch 3/200
 LR: 0.0010000000 
32000/32000 [==============================] - 399s 12ms/step - loss: 0.0335 - coef_determination: 0.5892 - val_loss: 0.1129 - val_coef_determination: -0.3911
Epoch 4/200
 LR: 0.0010000000 
32000/32000 [==============================] - 403s 13ms/step - loss: 0.0205 - coef_determination: 0.7488 - val_loss: 0.0490 - val_coef_determination: 0.3949
Epoch 5/200
 LR: 0.0010000000 
32000/32000 [==============================] - 405s 13ms/step - loss: 0.0120 - coef_determination: 0.8522 - val_loss: 0.0967 - val_coef_determination: -0.1910
Epo

In [46]:
history1_proper_simple_predict = model1_proper_simple.predict(X['test'])

In [49]:
predictions = history1_proper_simple_predict

### PRINT SCORE
# if all4:  
print( 'R2: ', 1 - (((predictions[:,0] - Y_basic['test'][:,0])**2).sum(axis=0)) / ((predictions[:,0] - predictions[:,0].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,1] - Y_basic['test'][:,1])**2).sum(axis=0)) / ((predictions[:,1] - predictions[:,1].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,2] - Y_basic['test'][:,2])**2).sum(axis=0)) / ((predictions[:,2] - predictions[:,2].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,3] - Y_basic['test'][:,3])**2).sum(axis=0)) / ((predictions[:,3] - predictions[:,3].mean(axis=0) )**2).sum(axis=0) )

np.save("LSTM1_proper_simple_predict.npy", predictions)

R2:  0.9958514552563429
R2:  0.999623172916472
R2:  0.9948521475307643
R2:  0.8878387436270714


In [222]:
for layer in model1_simple.layers: print(layer.get_config(), layer.get_weights())

{'name': 'Conv0', 'trainable': True, 'batch_input_shape': (None, 526, 25, 25, 1), 'dtype': 'float32', 'layer': {'class_name': 'Conv2D', 'config': {'name': 'conv2d_76', 'trainable': True, 'filters': 128, 'kernel_size': (11, 11), 'strides': (1, 1), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}} [array([[[[-3.29025269e-01, -5.21262050e-01, -5.43405235e-01, ...,
           6.07972026e-01,  2.50459909e-01, -3.53870600e-01]],

        [[-3.09221357e-01, -5.07967889e-01, -5.15023112e-01, ...,
           6.02953792e-01,  2.19846413e-01, -3.05888653e-01]],

        [[-2.24287063e-01, -4.825

In [10]:
model2 = Sequential()
model2.add(ConvLSTM2D(filters=32, kernel_size=(8, 8),
                   input_shape=input_shape, return_sequences=True))
# model2.add(ConvLSTM2D(filters=32, kernel_size=(5, 5),
#                    input_shape=input_shape, return_sequences=True))
model2.add(BatchNormalization())
model2.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2)), name="MaxPool"))
for i in range(2):
    model2.add(ConvLSTM2D(filters=64, kernel_size=(5, 5), return_sequences=True))
    model2.add(BatchNormalization())
model2.add(TimeDistributed(Flatten()))
model2.add(Dense(4, name="out"))
model2.summary()

opt = keras.optimizers.RMSprop(lr=0.001)
model2.compile( optimizer=opt, 
                loss=Loss1,
                metrics=[CoeffDetermination(1).timestep_cd, CoeffDetermination(1).timestep_loss])






_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_1 (ConvLSTM2D)  (None, 526, 18, 18, 32)   270464    
_________________________________________________________________
batch_normalization_1 (Batch (None, 526, 18, 18, 32)   128       
_________________________________________________________________
MaxPool (TimeDistributed)    (None, 526, 9, 9, 32)     0         
_________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)  (None, 526, 5, 5, 64)     614656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 526, 5, 5, 64)     256       
_________________________________________________________________
conv_lst_m2d_3 (ConvLSTM2D)  (None, 526, 1, 1, 64)     819456    
_________________________________________________________________
batch_normalization_3 (Batch (None, 526, 1, 1, 64)     256       
_____

In [15]:
history2 = model2.fit(X['train'], Y['train'],
                             epochs=200,
                             batch_size=20,
                             callbacks=callbacks_list,
                             validation_data=(X['val'], Y['val']),
                             verbose=True,
                             )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0010000000 
32000/32000 [==============================] - 17185s 537ms/step - loss: 2.8032e-04 - timestep_cd: 0.5576 - timestep_loss: 0.1567 - val_loss: 2.7113e-04 - val_timestep_cd: 0.5695 - val_timestep_loss: 0.1692
Epoch 2/200
 LR: 0.0010000000 
32000/32000 [==============================] - 17190s 537ms/step - loss: 2.3574e-04 - timestep_cd: 0.6281 - timestep_loss: 0.1837 - val_loss: 2.3282e-04 - val_timestep_cd: 0.6235 - val_timestep_loss: 0.2023
Epoch 3/200
 LR: 0.0010000000 
32000/32000 [==============================] - 17268s 540ms/step - loss: 2.1759e-04 - timestep_cd: 0.6573 - timestep_loss: 0.2053 - val_loss: 2.1226e-04 - val_timestep_cd: 0.6609 - val_timestep_loss: 0.2261
Epoch 4/200
 LR: 0.0010000000 
32000/32000 [==============================] - 17234s 539ms/step - loss: 1.9893e-04 - timestep_cd: 0.6857 - timestep_loss: 0.2188 - val_loss: 2.0174e-04 - val_timestep_cd: 0.6843 - val_timestep_loss: 0.2350

KeyboardInterrupt: 

In [ ]:
model2_

In [32]:
model_default = Sequential([
    Conv3D(64, kernel_size=(5, 8, 8), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Conv3D(64, kernel_size=(5, 5, 5), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 1, 1)),
    Conv3D(64, kernel_size=(5, 5, 5), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 1, 1)),
    Flatten(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4)
    ]
    )
# model_default.add(
# model_default.add()
# model_default.add(MaxPooling3D(pool_size=(2, 2, 2)))
# for i in range(2):
#     model_default.add(Conv3D(fliters=64, kernel_size=(5, 5, 5), activation='relu')
#     model_default.add(BatchNormalization())
# model_default.add(Flatten())
# model_default.add(Dropout(0.2))
# model_default.add(Dense(128, activation='relu'))
# model_default.add(Dense(64, activation='relu'))
# model_default.add(Dense(16, activation='relu'))
# model_default.add(Dense(4))
model_default.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 522, 18, 18, 64)   20544     
_________________________________________________________________
batch_normalization_13 (Batc (None, 522, 18, 18, 64)   256       
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 261, 9, 9, 64)     0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 257, 5, 5, 64)     512064    
_________________________________________________________________
batch_normalization_14 (Batc (None, 257, 5, 5, 64)     256       
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 128, 5, 5, 64)     0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 124, 1, 1, 64)     512064    
__________

In [38]:
opt = keras.optimizers.RMSprop(lr=0.1)
model_default.compile( optimizer=opt, 
                loss='mse',
                metrics=[coef_determination])

In [45]:
history_default = model_default.fit( X['train'], Y_basic['train'],
                                     epochs=200,
                                     batch_size=20,
                                     callbacks=callbacks_list,
                                     validation_data=(X['val'], Y_basic['val']),
                                     verbose=True,
                                     )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0003906250 
32000/32000 [==============================] - 776s 24ms/step - loss: 0.0829 - coef_determination: -0.0136 - val_loss: 0.0827 - val_coef_determination: -0.0128
Epoch 2/200
 LR: 0.0003906250 
32000/32000 [==============================] - 761s 24ms/step - loss: 0.0829 - coef_determination: -0.0137 - val_loss: 0.0827 - val_coef_determination: -0.0129
Epoch 3/200
 LR: 0.0003906250 
32000/32000 [==============================] - 611s 19ms/step - loss: 0.0829 - coef_determination: -0.0130 - val_loss: 0.0828 - val_coef_determination: -0.0138
Epoch 4/200
 LR: 0.0003906250 
32000/32000 [==============================] - 1061s 33ms/step - loss: 0.0829 - coef_determination: -0.0137 - val_loss: 0.0828 - val_coef_determination: -0.0135
Epoch 5/200
 LR: 0.0003906250 
32000/32000 [==============================] - 1354s 42ms/step - loss: 0.0829 - coef_determination: -0.0133 - val_loss: 0.0827 - val_coef_determination: -0

KeyboardInterrupt: 

In [42]:
predictions = model_default.predict( X['test'], verbose=True )

4000/4000 [==============================] - 118s 30ms/step


NameError: name 'testY' is not defined

In [43]:
print( 'R2: ', 1 - (((predictions[:,0] - Y_basic['test'][:,0])**2).sum(axis=0)) / ((predictions[:,0] - predictions[:,0].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,1] - Y_basic['test'][:,1])**2).sum(axis=0)) / ((predictions[:,1] - predictions[:,1].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,2] - Y_basic['test'][:,2])**2).sum(axis=0)) / ((predictions[:,2] - predictions[:,2].mean(axis=0) )**2).sum(axis=0) )
print( 'R2: ', 1 - (((predictions[:,3] - Y_basic['test'][:,3])**2).sum(axis=0)) / ((predictions[:,3] - predictions[:,3].mean(axis=0) )**2).sum(axis=0) )

R2:  -23221428551679.0
R2:  -22885194268671.0
R2:  -6087514259455.0
R2:  -23424650969087.0


In [53]:
model_default2 = Sequential([
    Conv3D(16, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(32, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(32, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(64, kernel_size=(10, 1, 1), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(64, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 1, 1)),
    Conv3D(64, kernel_size=(10, 1, 1), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(64, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 1, 1)),
    Conv3D(64, kernel_size=(10, 1, 1), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    Conv3D(64, kernel_size=( 1, 5, 5), input_shape=input_shape, activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 1, 1)),
    Flatten(),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dense(64, activation='relu'),
    Dense(4)
    ]
    )
model_default2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 526, 21, 21, 16)   416       
_________________________________________________________________
batch_normalization_37 (Batc (None, 526, 21, 21, 16)   64        
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 526, 17, 17, 32)   12832     
_________________________________________________________________
batch_normalization_38 (Batc (None, 526, 17, 17, 32)   128       
_________________________________________________________________
conv3d_39 (Conv3D)           (None, 526, 13, 13, 32)   25632     
_________________________________________________________________
batch_normalization_39 (Batc (None, 526, 13, 13, 32)   128       
_________________________________________________________________
conv3d_40 (Conv3D)           (None, 517, 13, 13, 64)   20544     
__________

In [54]:
opt = keras.optimizers.RMSprop(lr=0.001)
model_default2.compile( optimizer=opt, 
                loss='mse',
                metrics=[coef_determination])

In [55]:
history_default = model_default2.fit(X['train'], Y_basic['train'],
                                     epochs=200,
                                     batch_size=20,
                                     callbacks=callbacks_list,
                                     validation_data=(X['val'], Y_basic['val']),
                                     verbose=True,
                                     )

Train on 32000 samples, validate on 4000 samples
Epoch 1/200
 LR: 0.0010000000 
32000/32000 [==============================] - 1047s 33ms/step - loss: 0.1047 - coef_determination: -0.3060 - val_loss: 86.1134 - val_coef_determination: -1063.3730
Epoch 2/200
 LR: 0.0010000000 
32000/32000 [==============================] - 1014s 32ms/step - loss: 0.0285 - coef_determination: 0.6514 - val_loss: 0.0856 - val_coef_determination: -0.0514
Epoch 3/200
 LR: 0.0010000000 
32000/32000 [==============================] - 1017s 32ms/step - loss: 0.0259 - coef_determination: 0.6832 - val_loss: 0.2473 - val_coef_determination: -2.0455
Epoch 4/200
 LR: 0.0010000000 
32000/32000 [==============================] - 1012s 32ms/step - loss: 0.0248 - coef_determination: 0.6958 - val_loss: 0.0363 - val_coef_determination: 0.5540
Epoch 5/200
 LR: 0.0010000000 
32000/32000 [==============================] - 1036s 32ms/step - loss: 0.0243 - coef_determination: 0.7028 - val_loss: 0.1847 - val_coef_determination: 

KeyboardInterrupt: 